In [12]:
import hopsworks
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("HOPSWORKS_KEY")

In [13]:
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/602262
Connected. Call `.close()` to terminate connection gracefully.


In [25]:
# Retrieve feature groups
movies_fg = fs.get_feature_group(
    name='raw_movies_info', 
    version=3,
)
movies_fg.show(5)

Finished: Reading data from Hopsworks, using Hive (2.27s) 


,budget,id,imdb_id,popularity,release_date,revenue,title,cast_popularity,crew_popularity,star_present
0,224056,284288,tt3001638,6.920,2014-09-05,15536328,Mary Kom,33.536,27.757,no
1,3900000,159434,tt0489486,2.854,2007-05-11,4400000,"Good Boy, Bad Boy",44.078,18.443,no
2,1000,1020324,,0.600,2023-08-09,1400,Loveju,0.600,0.600,no
3,3213790,456570,tt5764096,12.610,2017-05-19,52390000,Hindi Medium,78.123,48.620,no
4,4500000,534780,tt8108198,18.665,2018-10-05,40000000,Andhadhun,66.013,58.827,no


In [26]:
query = movies_fg.select_all()
# Get or create the 'transactions_view_fraud_batch_fv' feature view
feature_view = fs.get_or_create_feature_view(
    name='movies_view',
    version=2,
    query=query,
    labels=["revenue"]
)

In [5]:
type(feature_view)

hsfs.feature_view.FeatureView

In [6]:
TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_size = TEST_SIZE,
)

Finished: Reading data from Hopsworks, using Hive (2.07s) 


In [7]:
X_train.columns

Index(['budget', 'id', 'imdb_id', 'popularity', 'release_date', 'title',
       'cast_popularity', 'crew_popularity', 'star_present'],
      dtype='object')

In [8]:
X_train.shape

(257, 9)

In [9]:
import pandas as pd
X_train = pd.get_dummies(X_train, columns=['star_present'])
X_test = pd.get_dummies(X_test, columns=['star_present'])

In [10]:
y_train

,revenue
0,15536328
1,4400000
4,40000000
5,108300000
6,6463160
...,...
317,5000000
318,62000000
319,34500000
320,10000000


In [11]:
train_data = pd.concat([X_train,y_train],axis=1)
train_data.head()

,budget,id,imdb_id,popularity,release_date,title,cast_popularity,crew_popularity,star_present_no,star_present_yes,revenue
0,224056,284288,tt3001638,6.920,2014-09-05,Mary Kom,33.536,27.757,True,False,15536328
1,3900000,159434,tt0489486,2.854,2007-05-11,"Good Boy, Bad Boy",44.078,18.443,True,False,4400000
4,4500000,534780,tt8108198,18.665,2018-10-05,Andhadhun,66.013,58.827,True,False,40000000
5,13000000,781732,tt13751694,51.221,2023-12-01,Animal,317.425,139.168,False,True,108300000
6,3400000,393421,tt5120640,4.855,2016-05-20,Sarbjit,39.602,19.357,True,False,6463160


In [39]:
import plotly.express as px

px.scatter(train_data,x='budget',y='revenue')

In [9]:
X_train.columns

Index(['budget', 'id', 'imdb_id', 'popularity', 'release_date', 'title',
       'cast_popularity', 'crew_popularity', 'star_present_no',
       'star_present_yes'],
      dtype='object')

In [12]:
features = ['budget', 'popularity','cast_popularity', 'crew_popularity', 'star_present_no','star_present_yes']
X_train_filtered = X_train[features]
X_test_filtered = X_test[features]

In [13]:
X_train_filtered

,budget,popularity,cast_popularity,crew_popularity,star_present_no,star_present_yes
0,224056,6.920,33.536,27.757,True,False
1,3900000,2.854,44.078,18.443,True,False
4,4500000,18.665,66.013,58.827,True,False
5,13000000,51.221,317.425,139.168,False,True
6,3400000,4.855,39.602,19.357,True,False
...,...,...,...,...,...,...
317,4500000,5.311,66.785,45.748,True,False
318,18000000,14.733,105.840,122.026,False,True
319,10000000,7.755,106.179,66.546,False,True
320,2300000,2.094,84.421,11.523,True,False


In [14]:
from sklearn.preprocessing import StandardScaler
import numpy as np
# Fit the scaler only on the training data
scaler = StandardScaler()
#X_train_filtered[['popularity','cast_popularity', 'crew_popularity']] = scaler.fit_transform(X_train_filtered[['popularity','cast_popularity', 'crew_popularity']])

# Use the same scaler to transform both training and testing data
#X_test_filtered[['popularity','cast_popularity', 'crew_popularity']] = scaler.transform(X_test_filtered[['popularity','cast_popularity', 'crew_popularity']])

# Apply log transformation to the column with range 0-300 million using the same scaler
X_train_filtered['budget'] = np.log1p(X_train['budget'])
X_test_filtered['budget'] = np.log1p(X_test['budget'])

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
y_train['revenue'] = np.log1p(y_train['revenue'])
y_test['revenue'] = np.log1p(y_test['revenue'])

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [42]:
model = LinearRegression().fit(X_train_filtered,y_train)
print(model.score(X_train_filtered,y_train))
y_pred=model.predict(X_test_filtered)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(mse,r2)

0.7687002812153442
2.954482962486932 0.7742970183433845


In [17]:
m1 = LinearRegression()
m2 = svm.SVR()
m3= DecisionTreeRegressor()
m4= GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0,loss='squared_error')
m5= RandomForestRegressor(random_state=1)
ereg = VotingRegressor(estimators=[('gb', m4), ('rf', m5), ('lr', m1)])

In [18]:
m1_fit = m5.fit(X_train_filtered,y_train)
y_pred = m1_fit.predict(X_test_filtered)
r2 = r2_score(y_test, y_pred)
print(r2)

0.8135718472254494


In [19]:
metrics = {
    "r2_score": r2_score(y_test, y_pred)
}
metrics

{'r2_score': 0.8135718472254494}

In [20]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Define the input schema using the values of X_train
input_schema = Schema(X_train_filtered.values)

# Define the output schema using y_train
output_schema = Schema(y_train)

# Create a ModelSchema object specifying the input and output schemas
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

# Convert the model schema to a dictionary for further inspection or serialization
model_schema.to_dict()

{'input_schema': {'tensor_schema': {'shape': '(257, 6)', 'type': 'object'}},
 'output_schema': {'columnar_schema': [{'name': 'revenue',
    'type': 'float64'}]}}

In [21]:
import joblib
# Specify the directory where the model will be saved
model_dir = "revenue_prediction_model"

# Check if the directory exists, and create it if it doesn't
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

# Save the trained  model using joblib
joblib.dump(m5, model_dir + '/random_forest_revenue_prediction.pkl')



['revenue_prediction_model/random_forest_revenue_prediction.pkl']

In [22]:
# Get the model registry
mr = project.get_model_registry()

# Create a new model in the model registry
fraud_model = mr.python.create_model(
    name="random_forest_revenue_prediction_model",     # Name for the model
    metrics=metrics,                      # Metrics used for evaluation
    model_schema=model_schema,            # Schema defining the model's input and output
    input_example=X_train_filtered.sample(),       # Example input data for reference
    description="Revenue Predictor",  # Description of the model
)

# Save the model to the specified directory
fraud_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/2344145 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/67 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/232 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/602262/models/random_forest_revenue_prediction_model/1


Model(name: 'random_forest_revenue_prediction_model', version: 1)

In [14]:
# Get the model registry
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [15]:
# Retrieve the model from the model registry
retrieved_model = mr.get_model(
    name="random_forest_revenue_prediction_model",
    version=1,
)

# Download the saved model files to a local directory
saved_model_dir = retrieved_model.download()

In [16]:
import joblib
# Load the saved XGBoost model using joblib from the downloaded model directory
retrieved_xgboost_model = joblib.load(saved_model_dir + "/random_forest_revenue_prediction.pkl")

# Display the retrieved XGBoost model
retrieved_xgboost_model

RandomForestRegressor(random_state=1)

In [27]:
# Initialize batch scoring
feature_view.init_batch_scoring(1)

# Get the batch data
batch_data = feature_view.get_batch_data()



# Display the first 3 rows
batch_data.head(3)

Finished: Reading data from Hopsworks, using Hive (2.05s) 


,budget,id,imdb_id,popularity,release_date,title,cast_popularity,crew_popularity,star_present
0,224056,284288,tt3001638,6.920,2014-09-05,Mary Kom,33.536,27.757,no
1,3900000,159434,tt0489486,2.854,2007-05-11,"Good Boy, Bad Boy",44.078,18.443,no
2,1000,1020324,,0.600,2023-08-09,Loveju,0.600,0.600,no


In [28]:
upcoming_movies

,budget,id,imdb_id,popularity,release_date,revenue,title,cast_popularity,crew_popularity,star_present
0,42105000,936622,tt18072316,31.783,2024-04-10,8.531093e+07,Bade Miyan Chote Miyan,101.524,71.525,yes
1,20000000,668340,tt10869778,23.633,2024-04-10,5.189607e+07,Maidaan,50.401,39.363,no


In [28]:
batch_data = pd.get_dummies(batch_data, columns=['star_present'])
features = ['budget', 'popularity','cast_popularity', 'crew_popularity', 'star_present_no','star_present_yes']
batch_data = batch_data[features]
batch_data['budget'] = np.log1p(batch_data['budget'])


In [2]:
%cd ..
from src import helpers 

/home/codespace/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/workspaces/box_office_prediction


In [6]:
import pandas as pd
import numpy as np

In [7]:
upcoming_movies = helpers.get_upcoming_movies_list()
batch_data = pd.get_dummies(upcoming_movies, columns=['star_present'])
features = ['budget', 'popularity','cast_popularity', 'crew_popularity', 'star_present_no','star_present_yes']
batch_data = batch_data[features]
batch_data['budget'] = np.log1p(batch_data['budget'])

In [17]:
# Use the retrieved XGBoost model to make predictions on the batch data
predictions = retrieved_xgboost_model.predict(batch_data)

# Display the first five predictions
predictions

array([18.26181318, 17.76475361])

In [20]:
upcoming_movies

,budget,id,imdb_id,popularity,release_date,revenue,title,cast_popularity,crew_popularity,star_present
0,42105000,936622,tt18072316,31.783,2024-04-10,0,Bade Miyan Chote Miyan,101.524,71.525,yes
1,20000000,668340,tt10869778,23.633,2024-04-10,0,Maidaan,50.401,39.363,no


In [21]:
upcoming_movies['revenue'] = np.expm1(predictions)

In [22]:
upcoming_movies

,budget,id,imdb_id,popularity,release_date,revenue,title,cast_popularity,crew_popularity,star_present
0,42105000,936622,tt18072316,31.783,2024-04-10,8.531093e+07,Bade Miyan Chote Miyan,101.524,71.525,yes
1,20000000,668340,tt10869778,23.633,2024-04-10,5.189607e+07,Maidaan,50.401,39.363,no


In [ ]:
pred_act['Predictions'] =np.expm1(pred_act['Predictions'])

In [51]:
X_train_filtered.head()

,budget,popularity,cast_popularity,crew_popularity,star_present_no,star_present_yes
0,12.319656,6.920,33.536,27.757,True,False
1,15.176487,2.854,44.078,18.443,True,False
2,6.908755,0.600,0.600,0.600,True,False
3,14.982962,12.610,78.123,48.620,True,False
4,15.319588,18.665,66.013,58.827,True,False


In [52]:
X_train.head()

,budget,id,imdb_id,popularity,release_date,title,cast_popularity,crew_popularity,star_present_no,star_present_yes
0,224056,284288,tt3001638,6.920,2014-09-05,Mary Kom,33.536,27.757,True,False
1,3900000,159434,tt0489486,2.854,2007-05-11,"Good Boy, Bad Boy",44.078,18.443,True,False
2,1000,1020324,,0.600,2023-08-09,Loveju,0.600,0.600,True,False
3,3213790,456570,tt5764096,12.610,2017-05-19,Hindi Medium,78.123,48.620,True,False
4,4500000,534780,tt8108198,18.665,2018-10-05,Andhadhun,66.013,58.827,True,False


In [53]:
y_train.head()

,revenue
0,16.558692
1,15.297115
2,7.244942
3,17.774226
4,17.504390


In [27]:
type(y_pred)

numpy.ndarray

In [28]:
type(y_test)

pandas.core.frame.DataFrame

In [60]:
pd.options.display.float_format = '{:,.2f}'.format
pred_act = pd.concat([pd.DataFrame(y_pred,columns=['Predictions']),y_test.reset_index(drop=True)],axis=1)
pred_act['Predictions'] =np.expm1(pred_act['Predictions'])
pred_act['revenue']=np.expm1(pred_act['revenue'])

In [62]:
pred_act['diff'] = pred_act['Predictions']-pred_act['revenue']

In [63]:
pred_act

,Predictions,revenue,diff
0,"17,084,159.25","46,000,000.00","-28,915,840.75"
1,"18,085,989.87","18,130,000.00","-44,010.13"
2,47.97,12.00,35.97
3,"14,306,096.70","256,884,693.00","-242,578,596.30"
4,"16,992,049.04","18,000,000.00","-1,007,950.96"
...,...,...,...
60,"12,076,068.96","5,200,000.00","6,876,068.96"
61,"13,222,775.21","67,000,000.00","-53,777,224.79"
62,"14,455,862.73","13,311,117.00","1,144,745.73"
63,"7,510,933.16","3,000,000.00","4,510,933.16"


In [64]:
X_test_filtered

,budget,popularity,cast_popularity,crew_popularity,star_present_no,star_present_yes
15,15.86,9.57,102.11,99.22,False,True
23,16.01,6.53,63.45,40.71,True,False
27,2.08,2.50,48.79,15.70,True,False
35,15.62,10.75,90.32,41.37,False,True
37,15.44,9.75,73.72,89.43,True,False
...,...,...,...,...,...,...
298,15.44,7.53,31.29,44.59,True,False
300,16.97,10.47,73.51,48.37,False,True
301,15.71,10.39,96.84,60.12,False,True
306,14.65,6.79,307.23,76.99,False,True


In [55]:
import helpers
helpers.get_features_movie(284288)

{'budget': 224056,
 'id': 284288,
 'imdb_id': 'tt3001638',
 'popularity': 13.378,
 'release_date': '2014-09-05',
 'revenue': 15536328,
 'title': 'Mary Kom',
 'cast_popularity': 59.541000000000004,
 'crew_popularity': 45.71200000000001,
 'star_present': 'no'}

In [59]:
import plotly.express as px

px.scatter(pred_act,x='revenue',y='Predictions')

In [ ]:
models = {
    'Linear Regression' : LinearRegression(),
    'SVM': svm.SVR(),
    'Decision Tree': DecisionTreeRegressor(),
    'Gradient Boost': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0,loss='squared_error'),

>>> ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])
>>> ereg = ereg.fit(X, y)





}

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

class RegressionModelEvaluator:
    def __init__(self, X_train,X_test, y_train,y_test):
        self.X_train, self.X_test, self.y_train, self.y_test = X_train,X_test, y_train,y_test
        self.models = {
            'Linear Regression': LinearRegression(),
            'Random Forest': RandomForestRegressor(random_state=42)
            # Add more models as needed
        }
        self.results = {}

    def fit_models(self):
        for name, model in self.models.items():
            model.fit(self.X_train, self.y_train)
            self.results[name] = {}

    def evaluate_models(self):
        for name, model in self.models.items():
            y_pred = model.predict(self.X_test)
            mse = mean_squared_error(self.y_test, y_pred)
            r2 = r2_score(self.y_test, y_pred)
            self.results[name]['MSE'] = mse
            self.results[name]['R-squared'] = r2

    def get_results(self):
        return self.results

# Example usage:
# Initialize RegressionModelEvaluator with your feature matrix X and target vector y
# evaluator = RegressionModelEvaluator(X, y)
# Fit models
# evaluator.fit_models()
# Evaluate models
# evaluator.evaluate_models()
# Get results
# results = evaluator.get_results()
# print(results)


In [22]:
evaluator = RegressionModelEvaluator(X_train_filtered,X_test_filtered,y_train,y_test)

In [23]:
evaluator.fit_models()

/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [24]:
evaluator.evaluate_models()

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- budget_transformed
